# Differentiable Neural Computer

![DNC architecture](https://storage.googleapis.com/deepmind-live-cms/images/dnc_figure1.width-1500_Zfxk87k.png "DNC architecture")

![DNC model](images/dnc_model.png "DNC model")


Paper from Deep Mind:
[Graves, Alex, et al. "Hybrid computing using a neural network with dynamic external memory." Nature 538.7626 (2016): 471-476.](https://www.nature.com/nature/journal/v538/n7626/abs/nature20101.html)

Deep Mind blog post:
[Differentiable neural computers](https://deepmind.com/blog/differentiable-neural-computers/)

Deep Mind implementation (Tensorflow + Sonnet):
[deepmind/dnc](https://github.com/deepmind/dnc)

This code was based in Siraj Raval [tutorial](https://www.youtube.com/watch?v=r5XKzjTFCZQ) and [code](https://github.com/llSourcell/differentiable_neural_computer_LIVE)

<div class="alert alert-info">
Every time step dependant operation will be defined inside a function, and shared weights and constants will be defined outside. All functions will receive a dictionary `tm1` ($t - 1$) with all the values from the previous step and another dictionary `t` with part of the information of the current step.
</div>

In [ ]:
import tensorflow as tf
import numpy as np

import datetime

from __future__ import print_function


tf.reset_default_graph() # for executing this block more than once

epsilon = 1e-6
normal_stddev = 0.1
t_normal_initializer = tf.truncated_normal_initializer(mean=0, stddev=normal_stddev) # initialization ~ N(0, 0.1)
t_epsilon_initializer = tf.constant_initializer(value=epsilon) # initialization with small positive constant
t_zeros_initializer = tf.zeros_initializer() # initialization with zero

## Definitions

$t \in \mathbb{N}$: time step

$X$: input domain

$Y$: output domain

$M_t \in \mathbb{R}^{N \times W}$: memory matrix at time $t$

$r_t^i \in \mathbb{R}^W$: read vector at time $t$ from read head $i$

$[v_1; \ldots; v_k]$: concatenation of $k$ vectors

$\circ$: elementwise multiplication

$\mathcal{S}_K$: $(K-1)$-dimensional unit simplex;
$\mathcal{S}_K = \{\alpha \in \mathbb{R}^K: \alpha_i \in [0, 1], \sum_{i=1}^K \alpha_i = 1\}$

$\Delta_K$: non-negative orthant of $\mathbb{R}^K$ with $\mathcal{S}_K$ as boundary;
$\Delta_K = \{\alpha \in \mathbb{R}^K: \alpha_i \in [0, 1], \sum_{i=1}^K \alpha_i \leq 1\}$

<div class="alert alert-info">
These numbers are problem dependant. In this example we will use sequences of 10 one hot encoding vectors of size 4.
</div>

In [ ]:
sequence_length = 6
sequence_width = 4

input_size = sequence_width # X
output_size = sequence_width # Y

### Activations

* $\sigma(x) = (1 + e^{-x})^{-1}$
* $oneplus(x) = 1 + softplus(x) = 1 + \log(1 + e^x)$
* $softmax(x)[i] = e^{x[i]} (\sum_{j=1}^n e^{x[j]})^{-1}$

## Hyperparameters

$N \in \mathbb{N}$: number of memory rows or locations

$W \in \mathbb{N}$: number of memory columns or word length

$R \in \mathbb{N}$: number of read heads

$L \in \mathbb{N}$: number of controller network layers

In [ ]:
num_words = 10 # N
word_size = 4 # W
num_read_heads = 1 # R
num_layers = 2 # L

## Inputs

$x_t \in \mathbb{R}^X$: input vector at time $t$

$z_t \in \mathbb{R}^Y$: target vector at time $t$

In [ ]:
t_inputs = tf.placeholder(tf.float32, [sequence_length * 2, input_size], name="x")
t_targets = tf.placeholder(tf.float32, [sequence_length * 2, output_size], name="z")

## Controller network

$\mathcal{N}$: controller network

$\mathcal{X}_t \in \mathbb{R}^{X + RW}$: controller input vector at time $t$; $\mathcal{X}_t = [x_t; r_{t-1}^1; \ldots, r_{t-1}^R]$

$\theta$: controller parameter vector (all the weights)

* Recurrent Neural Network: $(\nu_t, \xi_t) = \mathcal{N}([\mathcal{X}_1; \ldots; \mathcal{X}_t]; \theta)$

* Feed-forward Neural Network: $(\nu_t, \xi_t) = \mathcal{N}(\mathcal{X}_t; \theta)$

Deep LSTM variant used in the paper:

$$i_t^l = \sigma(W_i^l[\mathcal{X}_t; h_{t - 1}^l; h_t^{l-1}] + b_i^l)$$

$$f_t^l = \sigma(W_f^l[\mathcal{X}_t; h_{t - 1}^l; h_t^{l-1}] + b_f^l)$$

$$s_t^l = f_t^l s_{t-1}^l + i_t^l tanh(W_s^l[\mathcal{X}_t; h_{t - 1}^l; h_t^{l-1}] + b_s^l)$$

$$o_t^l = \sigma(W_o^l[\mathcal{X}_t; h_{t - 1}^l; h_t^{l-1}] + b_o^l)$$

$$h_t^l = o_t^l tanh(s_t^l)$$

Where for layer $l$ at time $t$:

$h_t^l$: hidden state

$i_t^l$: input gate

$f_t^l$: forget gate

$s_t^l$: state gate

$o_t^l$: output gate

$h_t^0 = 0$, $h_0^l = 0$ and $s_0^l = 0$.

<div class="alert alert-info">
__We will use a simple feed-forward network of two layers__

So the value $[h_t^1; \ldots, h_t^L]$ will be replaced by last activation of the network
</div>

In [ ]:
fc_hidden1_size = 32

interface_size = word_size * num_read_heads + 3 * word_size + 5 * num_read_heads + 3
controller_input_size = input_size + num_read_heads * word_size
controller_output_size = output_size + interface_size

with tf.variable_scope("N"):
    with tf.variable_scope("fc1"):
        t_W1 = tf.get_variable(
            "W", [controller_input_size, fc_hidden1_size], dtype=np.float32, initializer=t_normal_initializer)

        t_b1 = tf.get_variable(
            "b", [fc_hidden1_size], dtype=np.float32, initializer=t_zeros_initializer)

    with tf.variable_scope("fc2"):
        t_W2 = tf.get_variable(
            "W", [fc_hidden1_size, controller_output_size], dtype=np.float32, initializer=t_normal_initializer)

        t_b2 = tf.get_variable(
            "b", [controller_output_size], dtype=np.float32, initializer=t_zeros_initializer)

def controller(tm1, t):
    with tf.variable_scope("N"):
        t["controller_input"] = tf.concat(
            [t["input"], tf.reshape(tm1["read"], [1, num_read_heads * word_size])], 1, name="X")

        with tf.variable_scope("fc1"):
            t_z1 = tf.add(tf.matmul(t["controller_input"], t_W1, name="Wa"), t_b1, name="z")

            t_a1 = tf.nn.tanh(t_z1, name="a")

        with tf.variable_scope("fc2"):
            t_z2 = tf.add(tf.matmul(t_a1, t_W2, name="Wa"), t_b2, name="z")

            t_a2 = tf.nn.tanh(t_z2, name="a")
            
    t["controller_activation"] = t_a2

## Calculate interface

$W_{\xi}$: hidden to interface weight matrix

$\xi_t \in \mathbb{R}^{WR + 3W + 5R + 3}$:
interface vector (interactions with the memory) at time $t$;
$\xi = W_{\xi}[h_t^1; \ldots, h_t^L]$

In [ ]:
t_hidden_to_interface = tf.get_variable(
    "W_xi", [controller_output_size, interface_size], dtype=np.float32, initializer=t_normal_initializer)

def calculate_interface(tm1, t):
    t["interface"] = tf.matmul(t["controller_activation"], t_hidden_to_interface, name="xi")

## Split the interface into parts

$\xi_t= [
k_t^{r,1}; \ldots; k_t^{r,R};
\hat{\beta}_t^{r,1}; \ldots; \hat{\beta}_t^{r,R};
k_t^w; \hat{\beta}_t^w; \hat{e}_t; v_t;
\hat{f}_t^1; \ldots; \hat{f}_t^R;
\hat{g}_t^a; \hat{g}_t^w;
\hat{\pi}_t^1; \ldots; \hat{\pi}_t^R
]$

$\{k_t^{r,i} \in \mathbb{R}^W; 1 \leq i \leq R\}$: read keys

$\{\beta_t^{r,i} = oneplus(\hat{\beta}_t^{r,i}) \in [1, \infty); 1 \leq i \leq R\}$: read strengths

$k_t^w \in \mathbb{R}^W$: write key

$\beta_t^w = oneplus(\hat{\beta}_t^w) \in [1, \infty)$: write strength

$e_t = \sigma(\hat{e}_t) \in [0, 1]^W$: erase vector

$v_t \in \mathbb{R}^W$: write vector

$\{f_t^i = \sigma(\hat{f}_t^i) \in [0, 1]; 1 \leq i \leq R\}$: free gates

$g_t^a = \sigma(\hat{g}_t^a) \in [0, 1]$: allocation gate

$g_t^w = \sigma(\hat{g}_t^w) \in [0, 1]$: write gate

$\{\pi_t^i = softmax(\hat{\pi}_t^i) \in \mathcal{S}_3; 1 \leq i \leq R\}$: read modes

In [ ]:
# the partition will be of size 10;
# this tensor indicates for each position the corresponding part
t_partition = tf.constant([
    [0] * (num_read_heads * word_size) + \
    [1] * (num_read_heads) + \
    [2] * (word_size) + \
    [3] + \
    [4] * (word_size) + \
    [5] * (word_size) + \
    [6] * (num_read_heads) + \
    [7] + \
    [8] + \
    [9] * (num_read_heads * 3)
], dtype=tf.int32)

def split_interface(tm1, t):
    (t_read_keys,
     t_read_strengths,
     t_write_key,
     t_write_strength,
     t_erase,
     t_write,
     t_free_gates,
     t_allocation_gate,
     t_write_gate,
     t_read_modes) = tf.dynamic_partition(t["interface"], t_partition, 10)

    # reshape and apply activations to each part
    
    with tf.variable_scope("k-r"):
        t["read_keys"] = tf.reshape(t_read_keys, [num_read_heads, word_size])

    with tf.variable_scope("beta-r"):
        t["read_strengths"] = tf.add(1.0, tf.nn.softplus(tf.expand_dims(t_read_strengths, 0)))

    with tf.variable_scope("k-w"):
        t["write_key"] = tf.expand_dims(t_write_key, 0)

    with tf.variable_scope("beta-w"):
        t["write_strength"] = tf.add(1.0, tf.nn.softplus(tf.expand_dims(t_write_strength, 0)))

    with tf.variable_scope("e"):
        t["erase"] = tf.nn.sigmoid(tf.expand_dims(t_erase, 0))

    with tf.variable_scope("v"):
        t["write"] = tf.expand_dims(t_write, 0)

    with tf.variable_scope("f"):
        t["free_gates"] = tf.nn.sigmoid(tf.expand_dims(t_free_gates, 0))

    t["allocation_gate"] = tf.nn.sigmoid(t_allocation_gate, name="g-a")

    t["write_gate"] = tf.nn.sigmoid(t_write_gate, name="g-w")

    with tf.variable_scope("pi"):
        t["read_modes"] = tf.nn.softmax(tf.reshape(t_read_modes, [3, num_read_heads]))

## Content-based addressing

Used for reading and writing and it is related to assosiative structures.

$$\mathcal{C}(M, k, \beta)[i] = \frac{exp\{\mathcal{D(k, M[i, \cdot])\beta}\}}
{(\sum_{j=i}^N exp\{\mathcal{D(k, M[j, \cdot])\beta}\})}$$

Where:

$\mathcal{C}(M, k, \beta) \in \mathcal{S}_N$ defines a normalized probability distribution over the memory locations,

$k \in \mathbb{R}^W$ is the lookup key,

$\beta \in [1, \infty)$ is the key strength,

and $\mathcal{D}$ is the cosine similarity defined as:

$$\mathcal{D}(u, v) = \frac{u \cdot v}{|u||v|}$$

In [ ]:
# this is function will be called for read and write
# so it is not defined like the rest
def content_lookup(t_memory, t_key, t_strength):
    with tf.variable_scope("C"):
        with tf.variable_scope("D"):
            t_normalized_memory = tf.nn.l2_normalize(t_memory, 1, name="M_norm")
            t_normalized_key = tf.nn.l2_normalize(t_key, 0, name="k_norm")
            t_similarity = tf.matmul(t_normalized_memory, t_normalized_key, transpose_b=True) 
        return tf.nn.softmax(t_similarity * t_strength, 0)

## Dynamic memory allocation

$\psi_t \in [0, 1]^N$: retention vector; represents by how much each location will not be freed by the free gates;
$\psi_t = \Pi_{i=1}^R (1- f_t^i w_{t-1}^{r, i})$

$u_t \in [0, 1]^N$: usage vector; $u_t = (u_{t-1} + w_{t-1}^w - u_{t-1} \circ w_{t-1}^w) \circ \psi_t$ where $u_0 = 0$

A location $i$ is used if it has been retained by the free gates ($\psi_t[i] \approx 1$), and were either already in use or have just been written to.

$\phi_t \in \mathbb{Z}^N$: free list; is defined by sorting the indices of the memory locations in ascending order of $u_t$

$a_t \in \Delta_N$: allocation weighting; provides new locations for writing:

$$a_t[\phi_t[j]] = (1 - u_t[\phi_t[j]]) \Pi_{i=1}^{j-1} u_t[\phi_t[i]]$$

If all usages are 1, then a $a_t = 0$ and the controller can no longer allocate memory without first freeing used locations.

The sort operation induces discontinuities at the points at which the sort order changes. The authors ignore these discontinuities when calculating the gradient, as they do not seem to be relevant to learning.

In [ ]:
def memory_allocation(tm1, t):
    with tf.variable_scope("psi"):
        t["retention"] = tf.reduce_prod(1 - t["free_gates"] * tm1["read_weighting"], reduction_indices=1)

    with tf.variable_scope("u"):
        t["usage"] = (tm1["usage"] + tm1["write_weighting"] - tm1["usage"] * tm1["write_weighting"]) * \
            t["retention"]
    
    with tf.variable_scope("phi"):
        # trick to sort in ascending mode:
        # negate the usage vector and select the top k positions,
        # with k equal to all the positions,
        # and then negate back to get the original usage vector with ascending sorting
        t_sorted_usage, t_free = tf.nn.top_k(-1 * t["usage"], k=num_words)
        t_sorted_usage *= -1

        t["free"] = t_free
    
    with tf.variable_scope("a"):
        ## TODO: review this part ##

        t_cumprod = tf.cumprod(t_sorted_usage, axis=0, exclusive=True)
        t_unorder = (1 - t_sorted_usage) * t_cumprod

        t_allocation_weighting = tf.zeros([num_words])

        t_identity = tf.constant(np.identity(num_words, dtype=np.float32))

        for pos, idx in enumerate(tf.unstack(t_free[0])):
            m = tf.squeeze(tf.slice(t_identity, [idx, 0], [1, -1]))
            t_allocation_weighting += m * t_unorder[0, pos]

        t_allocation_weighting = tf.reshape(t_allocation_weighting, [num_words, 1])

        ## -- ##
    
    t["allocation_weighting"] = t_allocation_weighting

## Calculate next state of memory

$c_t^w \in \mathcal{S}_N$: write content weighting;
$c_t^w = \mathcal{C}(M_{t - 1}, k_t^w, \beta_t^w)$

$w_t^w \in \Delta_N$: write weighting;
$w_t^w = g_t^w[g_t^a a_t + (1 - g_t^a) c_t^w]$

$M_t = M_{t - 1} \circ (E - w_t^w e_t^{\top}) + w_t^w v_t^{\top}$

Where $E$ is an $N \times W$ matrix of ones.

In [ ]:
def write_memory(tm1, t):
    with tf.variable_scope("c-w"):
        t["write_content_weighting"] = content_lookup(tm1["memory"], t["write_key"], t["write_strength"])
    
    with tf.variable_scope("w-w"):
        t["write_weighting"] = t["write_gate"] * \
            (t["allocation_gate"] * t["allocation_weighting"] + \
             (1 - t["allocation_gate"]) * t["write_content_weighting"])
    
    with tf.variable_scope("M"):
        t["memory"] = tm1["memory"] * (1 - tf.matmul(t["write_weighting"], t["erase"])) + \
            tf.matmul(t["write_weighting"], t["write"])

## Temporal memory linkage

$p_t \in \Delta_N$: precedence weighting;
$p_t = \left(1 - \sum_i w_t^w[i]\right) p_{t - 1} + w_t^w$ where $p_0 = 0$

$p_t[i]$ represents the degree to which location $i$ was the last one written to.

$L \in [0, 1]^{N \times N}$: temporal link matrix; where:

$$L_0[i, j] = 0, 1 \leq i, j \leq N$$

$$L_t[i, i] = 0, 1 \leq i \leq N$$

$$L_t[i, j] = (1 - w_t^w[i] - w_t^w[j]) L_{t - 1}[i, j] + w_t^w[i] p_{t - 1}[j]$$

$L[i, \cdot] \in \Delta_N$ and $L[\cdot, j] \in \Delta_N$ for all $1 \leq i, j \leq N$.

If $L[i, j] \approx 1$ then $i$ was written after $j$, otherwise $L[i, j] \approx 0$.

$Lw$ smoothly shifts the focus forwards to the locations written after those emphasized in $w$,
whereas $L^{\top}w$ shifts the focus backwards.

There is a sparse version of this matrix in the parper.

In [ ]:
def memory_linkage(tm1, t):
    with tf.variable_scope("L"):
        # trick to vectorize the calculation: repeat the write in N columns
        t_write_weighting_repeat = tf.matmul(t["write_weighting"], tf.ones([1, num_words]))

        t["link"] = (1 - t_write_weighting_repeat - tf.transpose(t_write_weighting_repeat)) * tm1["link"] + \
            tf.matmul(t["write_weighting"], tm1["precedence_weighting"], transpose_b=True)

        # trick to vectorize the calculation:
        # multiply by a matrix of ones with zeros in the diagonal
        # to turn into zeros the diagonal of the link matrix
        t["link"] *= tf.ones([num_words, num_words]) - tf.constant(np.identity(num_words, dtype=np.float32))
    
    with tf.variable_scope("p"):
        t["precedence_weighting"] = (1 - tf.reduce_sum(t["write_weighting"], reduction_indices=0)) * \
            tm1["precedence_weighting"] + t["write_weighting"]

## Calculate read vectors

For each read head $i$ with $1 \leq i \leq R$:

$c_t^{r, i} \in \mathcal{S}_N$: read content weighting;
$c_t^{r, i} = \mathcal{C}(M_{t - 1}, k_t^{r, i}, \beta_t^{r, i})$

$f_t^i \in \Delta_N$: forward weighting; $f_t^i = L_t w_{t - 1}^{r, i}$ *NOTE: the authors used the same symbol for the free gates!*

$b_t^i \in \Delta_N$: backward weighting; $b_t^i = L_t^{\top} w_{t - 1}^{r, i}$

$w^{r, i} \in \Delta_N$: read weighting;
$w^{r, i} = \pi_t^i[1] b_t^i + \pi_t^i[2] c_t^{r, i} + \pi_t^i[3] f_t^i$

$r_t^i$: read vector; $r_t^i = M_t^{\top} w_t^{r, i}$

If $\pi_t^i[2]$ dominates the read mode, then the weighting reverts to content lookup using $k_t^{r, i}$.

If $\pi_t^i[3]$ dominates, then the read head iterates through memory locations in the order they were written, ignoring the read key.

If $\pi_t^i[1]$ dominates, then the read head iterates in the reverse order.

In [ ]:
def calculate_read(tm1, t):
    with tf.variable_scope("c-r"):
        t["read_content_weighting"] = content_lookup(tm1["memory"], t["read_keys"], t["read_strengths"])
    
    t["forward_weighting"] = tf.matmul(t["link"], tm1["read_weighting"], name="fw")
    t["backwards_weighting"] = tf.matmul(t["link"], tm1["read_weighting"], transpose_a=True, name="b")

    with tf.variable_scope("w-r"):
        t["read_weighting"] = t["read_modes"][0] * t["backwards_weighting"] + \
            t["read_modes"][1] * t["read_content_weighting"] + \
            t["read_modes"][2] * t["forward_weighting"]
        
    t["read"] = tf.matmul(t["memory"], t["read_weighting"], transpose_a=True, name="r")

## Calculate output

$W_y$: hidden to output weight matrix

$W_r \in \mathbb{R}^{RW \times Y}$: read vectors to output weight matrix

$\nu_t \in \mathbb{R}^Y$: controller output vector at time $t$;
$\nu_t = W_y[h_t^1; \ldots, h_t^L]$

$y_t \in \mathbb{R}^Y$: output vector at time $t$;
$y_t = \nu_t + W_r[r_t^1; \ldots, r_t^R]$

In [ ]:
t_hidden_to_output = tf.get_variable(
    "W_y", [controller_output_size, output_size], dtype=np.float32, initializer=t_normal_initializer)

t_read_to_output = tf.get_variable(
    "W_r", [num_read_heads * word_size, output_size], dtype=np.float32, initializer=t_normal_initializer)

def calculate_output(tm1, t):
    t["controller_output"] = tf.matmul(t["controller_activation"], t_hidden_to_output, name="nu")
    
    t["output"] = tf.add(
        t["controller_output"],
        tf.matmul(tf.reshape(t["read"], [1, num_read_heads * word_size]), t_read_to_output),
        name="y")

## Initial step

According to the paper: $u_0 = 0$ and $p_0 = 0$. For $M_0$, $r_0$, $w_0^w$, $L_0$ and $w_0^r$ it is not clear.

In [ ]:
def initial_step():
    with tf.variable_scope("t0"):
        return {
            "memory": tf.zeros([num_words, word_size], name="M"),

            "read": tf.fill([num_read_heads, word_size], epsilon, name="r"),

            "usage": tf.zeros([num_words, 1], name="u"),

            "write_weighting": tf.fill([num_words, 1], epsilon, name="w-w"),

            "precedence_weighting": tf.zeros([num_words, 1], name="p"),

            "link": tf.zeros([num_words, num_words], name="L"),

            "read_weighting": tf.fill([num_words, num_read_heads], epsilon, name="w-r"),
        }

## Wrap one step of the model

In [ ]:
def dnc_step(tm1, t):
    controller(tm1, t)
    calculate_interface(tm1, t)
    split_interface(tm1, t)
    memory_allocation(tm1, t)
    write_memory(tm1, t)
    memory_linkage(tm1, t)
    calculate_read(tm1, t)
    calculate_output(tm1, t)

## Unroll the whole model

In [ ]:
def dnc():
    tm1 = initial_step()
    outputs = []
    for step, t_input in enumerate(tf.unstack(t_inputs, axis=0)):
        t_input = tf.expand_dims(t_input, 0)
        t = {"input": t_input}
        with tf.variable_scope("t" + str(step + 1)):
            dnc_step(tm1, t)
        tm1 = t
        outputs.append(t["output"])
    return tf.stack(outputs, axis=0)

In [ ]:
t_outputs = tf.squeeze(dnc())

t_loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=t_outputs, labels=t_targets))

regularization = 5e-4
regularizers = (tf.nn.l2_loss(t_W1) + tf.nn.l2_loss(t_W2) + tf.nn.l2_loss(t_b1) + tf.nn.l2_loss(t_b2))

t_loss += regularization * regularizers

learning_rate = 0.001
t_optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(t_loss)

In [ ]:
one_indices = np.random.randint(0, sequence_width, size=sequence_length)
sequence = np.zeros((sequence_length, sequence_width))
sequence[np.arange(sequence_length), one_indices] = 1

empty_sequence = np.zeros((sequence_length, sequence_width))

inputs = np.concatenate((sequence, empty_sequence), axis=0)
targets = np.concatenate((empty_sequence, sequence), axis=0)

In [ ]:
iterations = 1000
log_every = 100

with tf.Session() as session:
    session.run(tf.global_variables_initializer())
    
    for iteration in range(iterations):
        feed_dict = {t_inputs: inputs, t_targets: targets}
        
        loss, _, predictions = session.run([t_loss, t_optimizer, t_outputs], feed_dict=feed_dict)
        
        if iteration % log_every == log_every - 1:
            print(iteration + 1, loss)
            
    for i in range(sequence_length):
        j = sequence_length + i
        print(np.argmax(inputs[i, :]), np.argmax(targets[j, :]), np.argmax(predictions[j, :]))